## Import Libraries

In [1]:
import os
from crewai import Agent, Task, Crew, Process
from crewai.flow.flow import Flow, start, listen  
from langchain_community.tools import DuckDuckGoSearchRun
import textwrap



##  Disable CrewAI's telemetry to prevent data collection and errors

In [2]:
os.environ["CREWAI_TELEMETRY"] = "false"
os.environ["CREWAI_TELEMETRY_DEBUG"] = "0"  

## Load the  LLM


In [3]:
from crewai import LLM


llm = LLM(
    model="groq/qwen/qwen3-32b",
    api_key=os.getenv("GROQ_API_KEY"),
    api_base="https://api.groq.com/openai/v1",
    temperature=0.2,
    max_tokens=1000,
    num_retries=5,
    request_timeout=90
)

## Load  DuckduckGo Search tool

In [4]:
import time
import random
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

@tool("DuckDuckGo Search")
def search_tool1(query: str) -> str:
    """Searches the web using DuckDuckGo."""
    time.sleep(1.0 + random.random()*0.5) 
    return DuckDuckGoSearchRun().run(query)



## Topic for the LLM

In [5]:
topic = "2024 Honda Civic review"

## Defining all Agents

In [6]:
# #  Agent 1: Reviewer A with an Edmunds bias
reviewer1 = Agent(
    role="Reviewer A (Edmunds bias)",
    goal="Search Edmunds for the topic and extract concise pros/cons.Only output the final answer without explaining the steps",
    backstory="Automotive editor who summarizes objectively.",
    tools=[search_tool1],
    llm=llm,
    allow_delegation=False, 
    verbose=False  

)
#  Agent 2: Reviewer B with a Car and Driver bias
reviewer2 = Agent(
    role="Reviewer B (Car and Driver bias)",
    goal="Search Car and Driver for the topic and extract concise pros/cons.Only output the final answer without explaining the steps",
    backstory="Performance-focused reviewer with balanced takes.",
    tools=[search_tool1],
    llm=llm,
    allow_delegation=False, 
    verbose=False
)
# Agent 3: Reviewer C with a Top Gear/other magazine bias
reviewer3 = Agent(
    role="Reviewer C (Top Gear / other mags bias)",
    goal="Search Top Gear or other reputable magazines and extract concise pros/cons.Only output the final answer without explaining the steps",
    backstory="Keeps things practical for buyers.",
    tools=[search_tool1],
    llm=llm,
    allow_delegation=False, 
    verbose=False
)

#  Agent 4: Final Synthesizer, to combine the findings
synthesizer = Agent(
    role="Final Synthesizer",
    goal="Merge the three reviewer notes into one clear, non-redundant verdict.Only output the final answer without explaining the steps",
    backstory="Senior editor who reconciles differing opinions.",
    tools=[],  # no web/tools; purely synthesis
    llm=llm,
    allow_delegation=False, 
    verbose=False
)


## Giving tasks descriptions 

In [7]:


# Tasks for the three retrievers
t1 = Task(
    description=f"Topic: {topic}\nSearch site:edmunds.com {topic}. "
                "Return 5 bullets + 1 verdict + 1-2 links (plain URLs).",
    agent=reviewer1,
    expected_output="5 bullets + verdict + links"
)
t2 = Task(
    description=f"Topic: {topic}\nSearch site:caranddriver.com {topic}. "
                "Return 5 bullets + 1 verdict + 1-2 links (plain URLs).",
    agent=reviewer2,
    expected_output="5 bullets + verdict + links"
)
t3 = Task(
    description=f"Topic: {topic}\nSearch (site:topgear.com OR site:autocar.co.uk OR site:whatcar.com) {topic}. "
                "Return 5 bullets + 1 verdict + 1-2 links (plain URLs).",
    agent=reviewer3,
    expected_output="5 bullets + verdict + links"
)

# Running  ONLY the three retrievers in PARALLEL
t1.async_execution = True
t2.async_execution = True
t3.async_execution = True

# Final summarizer task (sequential) that COMBINES t1+t2+t3
t_final = Task(
    description=(
        "Combine the three reviewer outputs (given in context) into:\n"
        "- 4-6 consensus bullets (no duplicates)\n"
        "- One verdict paragraph (3-5 sentences) for buyers\n"
        "- 'Where reviewers agree' and 'Where they differ' (1-2 bullets each)\n"
        "Keep it neutral, specific, and text-only."
    ),
    agent=synthesizer,           #  4th agent
    expected_output="Bullets + verdict paragraph + agreement/differences",
    context=[t1, t2, t3]         # this pulls outputs from the three retrievers
)
from IPython.display import display, Markdown

# One crew: 3 parallel retrievers  final sequential summarizer
crew = Crew(
    agents=[reviewer1, reviewer2, reviewer3, synthesizer],
    tasks=[t1, t2, t3, t_final],   # final LAST
    process=Process.sequential,
    verbose=False,
)

result = crew.kickoff()

# Showing outputs 
outs = getattr(result, "tasks_output", []) or []
for i, t in enumerate(outs):
    role = getattr(getattr(t, "agent", None), "role", f"Task {i}")
    body = getattr(t, "raw", None) or getattr(t, "output", None) or ""
    display(Markdown(f"### {i}: {role}\n\n{body}"))

### 0: Task 0

- Fuel-efficient with a 30-mpg highway rating and available hybrid model
- Spacious interior with supportive seats and modern 12.3-inch digital gauge cluster
- Smooth, refined ride with precise steering and agile handling
- User-friendly infotainment system with Apple CarPlay and Android Auto
- Competitive starting price with strong resale value

Verdict: The 2024 Honda Civic delivers excellent fuel economy, a tech-rich interior, and agile handling, making it a top compact sedan choice for drivers who prioritize efficiency and driving dynamics.

https://www.edmunds.com/2024-honda-civic-review-rating.html
https://www.edmunds.com/honda/civic/

### 1: Task 1

**2024 Honda Civic Review (Car and Driver)**  
- **Pros**: Punchy 1.5T turbo engine delivers 182 hp with smooth 10-speed automatic; EPA-estimated 32 mpg highway.  
- **Cons**: Firm suspension prioritizes handling over comfort; base trims lack advanced safety tech.  
- **Tech**: 7-inch digital gauge cluster, 9-inch infotainment with Apple CarPlay/Android Auto, and wireless charging.  
- **Driving Dynamics**: Agile steering and precise handling make it a standout in its class.  
- **Value**: Competitive pricing with a well-equipped EX trim offering most desired features.  
**Verdict**: A well-rounded compact sedan that balances performance, efficiency, and tech—ideal for daily drivers seeking reliability.  
**Links**:  
https://www.caranddriver.com/honda/civic/reviews/2024-honda-civic-first-drive-review  
https://www.caranddriver.com/honda/civic/reviews/2024-honda-civic-1-5t-review

### 2: Task 2

**2024 Honda Civic Review**  
- **Pros**: Sharp handling and responsive steering; refined 1.5T turbo engine with smooth power delivery; high-quality interior with tech-rich infotainment; excellent fuel efficiency (up to 40 mpg); strong residual value.  
- **Cons**: Fidgety ride on rough roads; cramped rear legroom for taller passengers; optional tech packages can inflate the price.  
- **Verdict**: A well-rounded compact car with sporty dynamics and practicality, though comfort trade-offs may bother some.  

Links:  
https://www.topgear.com/car-reviews/honda/civic-2024-review  
https://www.whatcar.com/reviews/honda/civic-2024-review

### 3: Task 3

**Consensus Bullets**  
- Fuel-efficient with 30–40 mpg highway and a hybrid option for enhanced efficiency.  
- Agile handling, precise steering, and sporty driving dynamics.  
- Tech-focused interior featuring Apple CarPlay/Android Auto and modern infotainment (9-inch in higher trims).  
- Competitive starting price, strong resale value, and well-equipped mid-trim EX model.  

**Verdict**  
The 2024 Honda Civic excels as a compact sedan with a balance of fuel efficiency, driving engagement, and tech features. Its 1.5T turbo engine delivers responsive power, while the refined interior and user-friendly infotainment cater to modern drivers. Though base trims lack advanced safety tech and rear legroom is limited for taller passengers, the Civic’s strong value proposition and sporty handling make it a top choice for those prioritizing performance and efficiency.  

**Where Reviewers Agree**  
- All praise the Civic’s fuel efficiency, driving dynamics, and tech features.  
- Consensus on the 1.5T engine’s smooth power delivery and the vehicle’s strong resale value.  

**Where They Differ**  
- Edmunds and Car and Driver highlight a spacious interior, while Top Gear notes cramped rear legroom.  
- Car and Driver emphasizes the firm suspension’s focus on handling, whereas Top Gear criticizes the ride quality on rough roads.

HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Max retries exceeded with url: /v1/traces (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x14dcac5f0>, 'Connection to telemetry.crewai.com timed out. (connect timeout=29.9999942779541)'))
